# 01 - Upstream QC

In [ ]:
from libraries import *
from parameters import *

In [ ]:
%load_ext rpy2.ipython

In [ ]:
os.getcwd()
os.chdir(projectDir)

## Parameters

In [ ]:
conf_mt_prefix = 'MT-' if par_species == 'human' else 'mt-'

***

## Read the sample sheet and data sets

Load the sample sheet storing the metadata of the samples. Index column must include the unique h5ad or h5 file name. "sample_name" and "raw" are two mandatory columns, which stands for the unique sample identifier and whether the samples are the raw counts or not (True or False). Additional columns can be possible batch effects such as library name or sequencing channel.

In [ ]:
conf_sample_sheet = pd.read_csv(Path(par_data_dir) / 'samples.csv').set_index('h5ad_or_h5_path')
conf_sample_sheet

In [ ]:
conf_n_samples = len(conf_sample_sheet)

conf_plotting_n_cols = 5
conf_plotting_width = 20
conf_plotting_height_per_row = 3.5

conf_plotting_n_rows = math.ceil(conf_n_samples/conf_plotting_n_cols)
conf_sample_features = conf_sample_sheet.columns.tolist()

In [ ]:
assert 'sample_name' in conf_sample_sheet.columns, 'Sample sheet must have a unique sample_name column'
assert 'raw' in conf_sample_sheet.columns, 'Sample sheet must have a raw (True or False) column'
assert conf_sample_sheet.sample_name.nunique() == conf_n_samples, 'Sample sheet must have a unique sample_name column'

## Tools for prefiltered data

In [ ]:
def emptydrops(adata, lower, niters, ignore, retain=None, seed=0, **kwargs):
    r('set.seed')(seed)
    du = importr('DropletUtils')
    ed = du.emptyDrops(adata.X.T,
                       lower=lower,
                       niters=niters,
                       ignore=ignore,
                       retain=retain,
                       **kwargs)
    fdr = r('`[[`')(ed, 'FDR')
    adata.obs['empty_drops_FDR'] = fdr

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fdr[fdr == 0.0] = 1.0/niters

        adata.obs['neg_log10_empty_drops_FDR'] = -np.log10(fdr)

## Store QC info and run tools on prefiltered data

In [ ]:
%%time

conf_samples = {}

for sample in tqdm(list(conf_sample_sheet.itertuples())):
    file = Path(par_data_dir) / sample.Index

    if str(sample.raw).lower() == 'true':
        ad = sc.read_10x_h5(file).copy()
    else:
        ad = sc.read(file)

    ad.var_names_make_unique()
    display(ad)
    

    for sample_feature in conf_sample_features:
        ad.obs[sample_feature] = sample._asdict()[sample_feature]

    if str(sample.raw).lower() == 'true':
        # EmptyDrops
        emptydrops(ad,
                   lower=par_empty_drops_lower_umi_cutoff,
                   niters=par_empty_drops_niters,
                   ignore=par_empty_drops_ignore_cutoff,
                   retain=par_empty_drops_retain)
    else:
        ad.obs['empty_drops_FDR'] = np.nan
        ad.obs['neg_log10_empty_drops_FDR'] = np.nan

    ad.obs['n_umis']  = ad.X.sum(1)
    ad.obs['n_genes'] = (ad.X != 0).sum(1).A1
    ad.obs['log10_n_umis'] = np.log10(ad.X.sum(1))
    ad.obs['log10_n_genes'] = np.log10((ad.X != 0).sum(1).A1)

    # Total UMI/gene cutoffs
    ad = ad[ad.obs.n_umis  > par_initial_umi_cutoff]
    ad = ad[ad.obs.n_genes > par_initial_gene_cutoff]
    display(ad)

    # Save barcode rank
    ad.obs['barcode_rank'] = scipy.stats.rankdata(-ad.obs['n_umis'])

    # MT
    mt_gene_mask = ad.var_names.str.startswith(conf_mt_prefix)
    assert mt_gene_mask.sum() > 0, 'Wrong mt prefix'
    ad.obs['mt_frac'] = ad.X[:, mt_gene_mask].sum(1).A1 / ad.obs['n_umis']

    conf_samples[sample.sample_name] = ad.copy()

## UMI/Gene Rank Plots

In [ ]:
f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                      figsize=(conf_plotting_width,
                               conf_plotting_n_rows*conf_plotting_height_per_row),
                      sharey=True, sharex=True)
axs = axs.flatten()

for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
    ax.scatter(ad.obs.barcode_rank.values,
               ad.obs.n_umis.values,
               s=1, alpha=0.5)

    ax.axhline(par_empty_drops_ignore_cutoff, color='red')
    ax.axhline(par_empty_drops_lower_umi_cutoff, color='red')

    ax.set_yscale('log')
    ax.set_xscale('log')

    ax.set_xlabel('Barcode rank')
    ax.set_ylabel('nUMIs')

    ax.set_title(sample)

plt.subplots_adjust()

## nGenes vs nUMIs and distributions

In [ ]:
f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                      figsize=(conf_plotting_width,
                               conf_plotting_n_rows*conf_plotting_height_per_row),
                      sharey=True, sharex=True)
axs = axs.flatten()

for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
    ax.scatter(ad.obs.n_umis.values+1,
               ad.obs.n_genes.values+1,
               alpha=0.5,
               s=0.1)
    ax.axvline(par_empty_drops_ignore_cutoff, color='red')
    ax.axvline(par_empty_drops_lower_umi_cutoff, color='red')

    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_title(sample)

    ax.set_xlabel('nUMIs')
    ax.set_ylabel('nGenes')


f.suptitle('UMIs vs genes', fontsize=16)
plt.subplots_adjust(top=0.9)

## UMI distr.

f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                      figsize=(conf_plotting_width,
                               conf_plotting_n_rows*conf_plotting_height_per_row),
                      sharey=True, sharex=True)
axs = axs.flatten()

for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
    ax.hist(ad.obs.log10_n_umis.values, bins=100)
    ax.axvline(np.log10(par_empty_drops_ignore_cutoff), color='red')
    ax.axvline(np.log10(par_empty_drops_lower_umi_cutoff), color='red')

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.set_title(sample)
    ax.set_xlabel('nUMIs (log10)')

f.suptitle('UMI distributions', fontsize=16)
plt.subplots_adjust(top=0.9)

## Gene distr.

f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                      figsize=(conf_plotting_width,
                               conf_plotting_n_rows*conf_plotting_height_per_row),
                      sharey=True, sharex=True)
axs = axs.flatten()

for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
    ax.hist(ad.obs.log10_n_genes.values, bins=100)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.set_title(sample)
    ax.set_xlabel('nGenes (log10)')

f.suptitle('Gene distributions', fontsize=16)
plt.subplots_adjust(top=0.9)

## nGenes vs nUMIs and distributions (with EmptyDrops)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

if (conf_sample_sheet.raw.astype(str).str.lower() == 'true').any():

    f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                          figsize=(conf_plotting_width,
                                   conf_plotting_n_rows*conf_plotting_height_per_row),
                          sharey=True, sharex=True)
    axs = axs.flatten()

    for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
        if np.any(np.isfinite(ad.obs.neg_log10_empty_drops_FDR)):
            idx = np.argsort(ad.obs.neg_log10_empty_drops_FDR)[::-1]
            ad = ad[idx]

            pc = ax.scatter(ad.obs.n_umis.values+1,
                            ad.obs.n_genes.values+1,
                            s=0.1,
                            c=ad.obs.neg_log10_empty_drops_FDR)
            ax.axvline(par_empty_drops_ignore_cutoff, color='red')
            ax.axvline(par_empty_drops_lower_umi_cutoff, color='red')

            ax.set_yscale('log')
            ax.set_xscale('log')
            ax.set_title(sample)

            ax.set_xlabel('nUMIs')
            ax.set_ylabel('nGenes')

            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            f.colorbar(pc, cax=cax, orientation='vertical')

    f.suptitle('UMIs vs genes (colored by EmptyDrops FDR)', fontsize=16)
    plt.subplots_adjust(wspace=0.3, hspace=0.3, top=0.9)

    ## UMI distr.

    f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                          figsize=(conf_plotting_width,
                                   conf_plotting_n_rows*conf_plotting_height_per_row),
                          sharey=True, sharex=True)
    axs = axs.flatten()

    for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
        if np.any(np.isfinite(ad.obs.neg_log10_empty_drops_FDR)):
            ad = ad[ad.obs.neg_log10_empty_drops_FDR > -np.log10(par_empty_drops_fdr_cutoff)]
            ax.hist(ad.obs.log10_n_umis.values, bins=100)
            ax.axvline(np.log10(par_empty_drops_ignore_cutoff), color='red')
            ax.axvline(np.log10(par_empty_drops_lower_umi_cutoff), color='red')

            ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
            ax.set_title(sample)
            ax.set_xlabel('nUMIs (log10)')

    f.suptitle('EmptyDrops-selected barcodes (UMI distributions)', fontsize=16)
    plt.subplots_adjust(top=0.9)

    # Gene distr.

    f, axs = plt.subplots(conf_plotting_n_rows, conf_plotting_n_cols,
                          figsize=(conf_plotting_width,
                                   conf_plotting_n_rows*conf_plotting_height_per_row),
                          sharey=True, sharex=True)
    axs = axs.flatten()

    for sample, ad, ax in tqdm(list(zip(conf_samples.keys(), conf_samples.values(), axs))):
        if np.any(np.isfinite(ad.obs.neg_log10_empty_drops_FDR)):
            ad = ad[ad.obs.neg_log10_empty_drops_FDR > -np.log10(par_empty_drops_fdr_cutoff)]
            x = ad.obs.log10_n_genes.values
            ax.hist(x, bins=100)

            ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
            ax.set_title(sample)
            ax.set_xlabel('nGenes (log10)')

    f.suptitle('EmptyDrops-selected barcodes (gene distributions)', fontsize=16)
    plt.subplots_adjust(top=0.9)

## Serialize the session

In [ ]:
k = None
var = None

with shelve.open('session_01.pkl', protocol=4, flag='n') as db:
    for k, var in globals().items():
        if k.startswith('par_') or k.startswith('conf_'):
            print(f'Storing {k}...')
            db[k] = var
db.close()